# Energy Reconstruction Using CNN - Both Charges and Cos(Zenith)

In [4]:
import numpy as np
import tensorflow as tf
import os
import matplotlib.pyplot as plt

from datetime import datetime

from tensorflow import keras
from keras import layers
from keras import models
from keras import callbacks

#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, Conv2D, Flatten
#from tensorflow.keras.callbacks import ModelCheckpoint

from data_tools import load_preprocessed, dataPrep, nameModel

simPrefix = os.getcwd()+'\\simdata'

In [2]:
tf.__version__

'2.6.0'

## Data Input

In [5]:
x, y = load_preprocessed(simPrefix, 'train')

Percentage of events with a NaN: 2.68


In [6]:
print(x.shape)
print(y.keys())
# each station has 2 tanks, each tank has 2 DOMs (high/log gain)
# each tank measures charge and time
# each station gives 2 charges and 2 times, 4 total pieces of data per station
# stations arranged in 10x10 square lattice, 2 corners of square unused
# charge measured in VEM, vertical equivalent muon

# 'dir' is true direction, rest of dir are reconstruted by simulations
# 'plane_dir' assumes shower is flat plane
# 'laputop_dir' performs likelihood analysis
# 'small_dir' compromises between plane and laputop

(549773, 10, 10, 4)
dict_keys(['comp', 'energy', 'dir', 'plane_dir', 'laputop_dir', 'small_dir'])


## Model Training

### Alpha Model
- Input: no charge merge, no time layers included, normalized data, combined with cosine of zenith angle
- Layers: Two convolutional layers for charge, then combined with zenith
- Output: Energy

In [7]:
# Name for model
key = 'q1q2cosZ'
i = 0
while(os.path.exists('models/model_{}.h5'.format(key+str(i)))):
    i = i + 1
key = key+str(i)
numepochs = 20
# Data preparation: no merging of charge (q), no time layers included (t=False), data normalized from 0-1
prep = {'q':None, 't':False, 'normed':True, 'reco':'plane', 'cosz':True}

In [ ]:
key

In [6]:
# Create model using functional API for multiple inputs
charge_input=keras.Input(shape=(10,10,2,))

conv1_layer = layers.Conv2D(64,kernel_size=3,padding='same',activation='relu')(charge_input)
batch1_layer = layers.BatchNormalization()(conv1_layer) # default -> axis = -1, 
drop1_layer = layers.Dropout(0.2)(batch1_layer)

conv2_layer = layers.Conv2D(32,kernel_size=3,padding='same',activation='relu')(conv1_layer)
batch2_layer = layers.BatchNormalization()(conv2_layer)
drop2_layer = layers.Dropout(0.2)(batch2_layer)

conv3_layer = layers.Conv2D(16, kernel_size=3, padding='same',activation="relu")(conv2_layer)

flat_layer = layers.Flatten()(conv3_layer)
zenith_input=keras.Input(shape=(1,))
concat_layer = layers.Concatenate()([flat_layer,zenith_input])
#output = layers.Dense(1)(concat_layer)

dense1_layer = layers.Dense(512,activation='relu')(concat_layer)
batch3_layer = layers.BatchNormalization()(dense1_layer)
drop3_layer = layers.Dropout(0.2)(batch3_layer)

dense2_layer = layers.Dense(512,activation='relu')(dense1_layer)
batch4_layer = layers.BatchNormalization()(dense2_layer)
drop4_layer = layers.Dropout(0.2)(batch4_layer)

dense3_layer = layers.Dense(512,activation="relu")(dense2_layer)

output = layers.Dense(1)(dense3_layer)

model = models.Model(inputs=[charge_input,zenith_input],outputs=output,name=key)

model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

## Old model used for reference
#model = Sequential(name=nameModel(prep, 'test'))  # Automatic naming for flexible assessment later
## Add model layers
#model.add(Conv2D(64, kernel_size=3, activation='relu', input_shape=(10,10,2)))
#model.add(Conv2D(32, kernel_size=3, activation='relu'))
#model.add(Flatten())
#model.add(Dense(1)) # No activation function for last layer of regression model

## Compile model
#model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae','mse'])

In [8]:
# Establish arrays to be trained on
x_i = dataPrep(x, y, **prep)

In [9]:
# 85/15 split for training/validation
energy = x_i[2]['energy']
comp = x_i[2]['comp']
theta, phi = x_i[2]['dir'].transpose()
nevents = len(energy)
trainCut = (np.random.uniform(size=nevents) < 0.85)
testCut = np.logical_not(trainCut)

In [10]:
energy.shape

(540381,)

In [11]:
x_i[1].shape

(540381,)

In [11]:
model.summary()

Model: "q1q2cosZ3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 10, 10, 2)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 10, 10, 64)   1216        input_1[0][0]                    
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 10, 10, 32)   18464       conv2d[0][0]                     
__________________________________________________________________________________________________
conv2d_2 (Conv2D)               (None, 10, 10, 16)   4624        conv2d_1[0][0]                   
__________________________________________________________________________________________

In [12]:
keras.utils.plot_model(model,"model.png")

('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')


In [13]:
# Train
csv_logger = callbacks.CSVLogger('models/{}'.format(key))
early_stop = callbacks.EarlyStopping(patience=10, restore_best_weights=True) # default -> val_loss
checkpoint = callbacks.ModelCheckpoint('models/model_%s.h5' % key,save_best_only=True)
callbacklist = [early_stop, csv_logger,checkpoint]
history = model.fit(
    x=[x_i[0],x_i[1]], y=energy, epochs=numepochs,validation_split=0.15,callbacks=callbacklist)

Epoch 1/20
14354/14354 [==============================] - 270s 19ms/step - loss: 0.0820 - mae: 0.1820 - mse: 0.0820 - val_loss: 0.0278 - val_mae: 0.1268 - val_mse: 0.0278
Epoch 2/20
14354/14354 [==============================] - 271s 19ms/step - loss: 0.0307 - mae: 0.1312 - mse: 0.0307 - val_loss: 0.0278 - val_mae: 0.1241 - val_mse: 0.0278
Epoch 3/20
14354/14354 [==============================] - 271s 19ms/step - loss: 0.0280 - mae: 0.1244 - mse: 0.0280 - val_loss: 0.0266 - val_mae: 0.1224 - val_mse: 0.0266
Epoch 4/20
14354/14354 [==============================] - 273s 19ms/step - loss: 0.0268 - mae: 0.1213 - mse: 0.0268 - val_loss: 0.0256 - val_mae: 0.1154 - val_mse: 0.0256
Epoch 5/20
14354/14354 [==============================] - 270s 19ms/step - loss: 0.0259 - mae: 0.1189 - mse: 0.0259 - val_loss: 0.0201 - val_mae: 0.1018 - val_mse: 0.0201
Epoch 6/20
14354/14354 [==============================] - 265s 18ms/step - loss: 0.0253 - mae: 0.1172 - mse: 0.0253 - val_loss: 0.0223 - val_mae:

In [16]:
# Save model to file for easy loading
## WHERE ARE YOU SAVING TO?
np.save('models/model_%s.npy' % key,prep)
model.save('models/model_%s.h5' % key)
f = open("results.txt", "a")
now = datetime.now()
f.write("{}\t{}\tepochs:{}\tloss:{},{}\n".format(
    now.strftime("%m/%d/%Y %H:%M:%S"),
    key,
    len(history.history['loss']),
    history.history['loss'][len(history.history['loss'])-1],
    history.history['val_loss'][len(history.history['loss'])-1]
))
f.close()